### Eigenvector

https://qiskit.github.io/qiskit-aer/tutorials/7_matrix_product_state_method.html

In [31]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile

In [32]:
# Construct quantum circuit
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)
circ.measure([0,1], [0,1])

# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(circ, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

{'11': 519, '00': 505}

In [33]:
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)

# Define a snapshot that shows the current state vector
circ.save_statevector(label='my_sv')
circ.save_matrix_product_state(label='my_mps')
circ.measure([0,1], [0,1])

# Execute and get saved data
tcirc = transpile(circ, simulator)
result = simulator.run(tcirc).result()
data = result.data(0)

#print the result data
data

{'counts': {'0x3': 542, '0x0': 482},
 'my_sv': Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
              0.70710678+0.j],
             dims=(2, 2)),
 'my_mps': ([(array([[1.-0.j, 0.-0.j]]), array([[0.-0.j, 1.-0.j]])),
   (array([[1.-0.j],
           [0.-0.j]]),
    array([[0.-0.j],
           [1.-0.j]]))],
  [array([0.70710678, 0.70710678])])}

### Eigenvector with VQE

Esto esta mal, estoy intentandolo pero no lo he conseguido

In [1]:
from qiskit_algorithms import VQE
from qiskit_nature.units import DistanceUnit
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

In [2]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
hamiltonian = problem.hamiltonian

In [3]:
mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

noiseless_estimator = AerEstimator(
    run_options={"shots": 1024}
)

mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

num_spatial_orbitals = int(fermionic_op.num_spin_orbitals/2)
# The tuple of the number of alpha- and beta-spin particles
num_particles = (1, 0)

ansatz = UCCSD(
        num_spatial_orbitals,
        num_particles,
        mapper,
        initial_state=HartreeFock(
            num_spatial_orbitals,
            num_particles,
            mapper,
        ),
    )

vqe = VQE(noiseless_estimator, ansatz, COBYLA())

result_vqe = vqe.compute_minimum_eigenvalue(operator=qubit_op)

In [12]:
result_vqe.

array([6.34589298])

In [4]:
result_vqe.eigenvalue

-1.248179864289066

In [5]:

ansatz.draw()

┌───────────────────┐
q_0: ┤0                  ├
     │                   │
q_1: ┤1                  ├
     │  EvolvedOps(t[0]) │
q_2: ┤2                  ├
     │                   │
q_3: ┤3                  ├
     └───────────────────┘

┌───────────────────┐┌───┐     ┌───┐     ┌───┐     ┌───┐     ┌───┐     »
q_0: ┤0                  ├┤ H ├──■──┤ H ├──■──┤ H ├──■──┤ H ├──■──┤ H ├──■──»
     │                   │└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐»
q_1: ┤1                  ├─────┤ X ├─────┤ X ├─────┤ X ├─────┤ X ├─────┤ X ├»
     │  EvolvedOps(t[0]) │     └───┘     └───┘     └───┘     └───┘     └───┘»
q_2: ┤2                  ├──────────────────────────────────────────────────»
     │                   │                                                  »
q_3: ┤3                  ├──────────────────────────────────────────────────»
     └───────────────────┘                                                  »
«     ┌───┐     
«q_0: ┤ H ├──■──
«     └───┘┌─┴─┐
«q_1: ─────┤ X ├
«          └───┘
«q_2: ──────────
«               
«q_3: ──────────
«

In [ ]:
# Construct quantum circuit
circ = ansatz
circ.h(0)
circ.cx(0, 1)
circ.measure([0,1], [0,1])

# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(circ, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [37]:
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)

# Define a snapshot that shows the current state vector
circ.save_statevector(label='my_sv')
circ.save_matrix_product_state(label='my_mps')
circ.measure([0,1], [0,1])

# Execute and get saved data
tcirc = transpile(circ, simulator)
result = simulator.run(tcirc).result()
data = result.data(0)

#print the result data
data

{'counts': {'0x3': 504, '0x0': 520},
 'my_sv': Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
              0.70710678+0.j],
             dims=(2, 2)),
 'my_mps': ([(array([[1.-0.j, 0.-0.j]]), array([[0.-0.j, 1.-0.j]])),
   (array([[1.-0.j],
           [0.-0.j]]),
    array([[0.-0.j],
           [1.-0.j]]))],
  [array([0.70710678, 0.70710678])])}